In [1]:
import pandas as pd
import re
from docx import Document
import os
import openai
from tqdm import tqdm

In [2]:
openai.api_key = os.getenv("OPENAI_GPT4_KEY")

In [27]:
def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return ' '.join(full_text)


In [28]:
def split_sentences(text, n):
    sentences = re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', text)
    chunks = [sentences[i:i + n] for i in range(0, len(sentences), n)]
    return chunks

In [29]:
def create_dataframe(file_path):
    text = read_docx(file_path)
    chunks = split_sentences(text, 4)
    df = pd.DataFrame({'completion': [' '.join(chunk) for chunk in chunks]})
    return df

In [31]:
def create_summaries(thoughts_df):
    thoughts = thoughts_df['completion'].tolist()
    summaries = []
    for thought in tqdm(thoughts):
        message=[
            {"role": "user", "content": """The following text enclosed within carrot brackets <> is a snippet of somebody's stream of consciousness. Please summarize what they are thinking about in one brief sentence written as an instruction that would cause the thought to be generated. For example, your sentence could be formatted as "Think about ...", format your response as just this sentence and nothing else.\n\n<""" + thought + ">"}
        ]
        summary = openai.ChatCompletion.create(model="gpt-4", messages=message)["choices"][0]["message"]["content"]
        summaries.append(summary)
    return summaries

In [32]:
file_path = 'Thoughts.docx'
thoughts_df = create_dataframe(file_path)
thoughts_df.insert(0, 'prompt', create_summaries(thoughts_df))

100%|██████████| 109/109 [05:37<00:00,  3.09s/it]


In [35]:
thoughts_df.to_csv('Thoughts_Train_File.csv', index=False)

In [36]:
# os.system("openai tools fine_tunes.prepare_data -f Thoughts_Train_File.csv")

In [1]:
# openai api fine_tunes.create -t Thoughts_Train_File_prepared.jsonl -m davinci
# openai api fine_tunes.get -i ft-b32V3ts3ipRPPEOcMjqKnvp6
# openai api fine_tunes.follow -i ft-b32V3ts3ipRPPEOcMjqKnvp6